In [ ]:
import os

%matplotlib notebook 
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interp

C_mu = 0.09
SYMTENSORDIM = 6

# Case

In [ ]:
foam_dir = 'foam_ensemble'
results_dir = 'results_ensemble_precondition'
# results_dir = 'results_ensemble'
sample_dir = os.path.join(results_dir, 'samples')
data_dir = 'data'

tinit = 0
tint = 0
tfinal = 200

flow = 0
regularization = False

plot_full_channel = False
plot_samples = True
plot_int = False
savefigs = False

H = 0.5

In [ ]:
# y
y = np.loadtxt(os.path.join(data_dir, 'y')) / H

# cost & weights
J = []
W = []
if regularization:
    J0 = []
for i in range(tfinal+1):
    iJ = np.loadtxt(os.path.join(results_dir, f'J.{i}'))
    J.append(iJ.tolist())
    iW = np.loadtxt(os.path.join(results_dir, f'w.{i}'))
    W.append(iW)
    if regularization:
        iJ0 = np.loadtxt(os.path.join(results_dir, f'J_org.{i}'))
        J0.append(iJ0.tolist())
W = np.array(W)
nweights = W.shape[1]

# truth 
g_t = np.zeros([len(y)])
g_t[:] = -C_mu * np.ones(y.shape) 
U_t = np.loadtxt(os.path.join(data_dir, 'UxFullField'))

# initial
g_0 = np.loadtxt(os.path.join(results_dir, f'g.{tinit}')) 
U_0 = np.loadtxt(os.path.join(results_dir, f'U.flow_{flow}.{tinit}'))
W_0 = np.loadtxt(os.path.join(results_dir, f'w.{tinit}'))
g_0_samps =  np.loadtxt(os.path.join(sample_dir, f'flow_0.samps_g_0_iter_{tinit}'))
H_0_samps = np.loadtxt(os.path.join(sample_dir, f'flow_0.samps_H_measurement_0_iter_{tinit}'))

# intermediate
if plot_int:
    g_i = np.loadtxt(os.path.join(results_dir, f'g.{tint}')) 
    U_i = np.loadtxt(os.path.join(results_dir, f'U.flow_{flow}.{tint}'))
    W_i = np.loadtxt(os.path.join(results_dir, f'w.{tint}'))
    g_i_samps =  np.loadtxt(os.path.join(sample_dir, f'flow_0.samps_g_0_iter_{tint}'))
    H_i_samps = np.loadtxt(os.path.join(sample_dir, f'flow_0.samps_H_measurement_0_iter_{tint}'))
    
    
# final
g_1 = np.loadtxt(os.path.join(results_dir, f'g.{tfinal}')) 
U_1 = np.loadtxt(os.path.join(results_dir, f'U.flow_{flow}.{tfinal}'))
W_1 = np.loadtxt(os.path.join(results_dir, f'w.{tfinal}'))
g_1_samps =  np.loadtxt(os.path.join(sample_dir, f'flow_0.samps_g_0_iter_{tfinal}'))
H_1_samps = np.loadtxt(os.path.join(sample_dir, f'flow_0.samps_H_measurement_0_iter_{tfinal}'))

nsamples = g_0_samps.shape[1]
print('Done')

# Results

### Cost

In [ ]:
fig, ax = plt.subplots()
ax.plot(J[:tfinal+1], '-')
ax.set_xlabel(r'iteration')
ax.set_ylabel(r'$J$')
ax.set_title(r'Cost $J$')
ax.set_ylim(0., np.max(J[:tfinal+1]))
ax.set_xlim(0., tfinal)
if savefigs:
    plt.savefig('results_cost.pdf')
    
fig, ax = plt.subplots()
ax.semilogy(J[:tfinal+1], '-')
ax.set_xlabel(r'iteration')
ax.set_ylabel(r'$J$')
ax.set_title(r'Cost $J$')
ax.set_xlim(0., tfinal)
if savefigs:
    plt.savefig('results_cost_log.pdf')
 
if regularization:
    fig, ax = plt.subplots()
    ax.plot(J0[:tfinal+1], '-')
    ax.set_xlabel(r'iteration')
    ax.set_ylabel(r'$J$')
    ax.set_title(r'Cost NoReg')
    ax.set_ylim(0., np.max(J0[:tfinal+1]))
    ax.set_xlim(0., tfinal)
    if savefigs:
        plt.savefig('results_cost_noreg.pdf')
        
    fig, ax = plt.subplots()
    ax.semilogy(J0[:tfinal+1], '-')
    ax.set_xlabel(r'iteration')
    ax.set_ylabel(r'$J$')
    ax.set_title(r'Cost NoReg')
    ax.set_xlim(0., tfinal)
    if savefigs:
        plt.savefig('results_cost_log_noreg.pdf')

### Weights (initial and final values)

In [ ]:
plt.figure()
plt.title(r'Weights $W$')
plt.xlabel(r'Initial Weight')
plt.ylabel(r'Final Weight')

line = [min(np.min(W_0), np.min(W_1)), max(np.max(W_0), np.max(W_1))]
plt.plot(line, [0., 0.], 'k--', alpha=0.5)
plt.plot([0., 0.], line, 'k--', alpha=0.5)

plt.plot(W_0, W_1, 'ko')
plt.plot(line,line, 'y-', alpha=0.5)

if savefigs:
    plt.savefig('results_weights.pdf')

### functions $g^{(i)}$

In [ ]:
def plotg(ax, i, gt, g0, g1, gi=None):
    if plot_samples:
        plt.plot(np.zeros(len(y)), y, 'k--')
        for isamp in range(nsamples):
            if plot_int:
                plt.plot(g_i_samps[:, isamp], y, 'c-', alpha=0.25, linewidth=0.5)
            plt.plot(g_0_samps[:, isamp], y, 'r-', alpha=0.25, linewidth=0.5)
            plt.plot(g_1_samps[:, isamp], y, 'g-', alpha=0.25, linewidth=0.5)
    ax.plot(g0, y, 'r-', label='Initial')
    if plot_int:
        ax.plot(gi, y, 'c-', label='Intermediate') 
    ax.plot(g1, y, 'g-o', fillstyle='none', label='Final')
    ax.plot(gt, y, 'k-', label='Truth')
    ax.set_xlabel(r'$g_{' f'{i}' r'}$')
    ax.set_ylabel(r'$x_2$')
    ax.set_ylim([0., 1.0])
    return ax


def plotg_double(ax, i, gt, g0, g1, gi=None):
    yy = np.concatenate([y,y+0.5])
    plt.plot(np.zeros(len(yy)), yy, 'k--')
    if plot_samples:
        for isamp in range(nsamples):
            g0s = g_0_samps[:, isamp]
            gg0 = np.concatenate([g0s, g0s[::-1]])
            g1s = g_1_samps[:, isamp]
            gg1 = np.concatenate([g1s, g1s[::-1]])
            if plot_int:
                gis = g_i_samps[:, isamp]
                ggi = np.concatenate([gis, gis[::-1]])
                plt.plot(ggi, yy, 'c-', alpha=0.25, linewidth=0.5)
            plt.plot(gg0, yy, 'r-', alpha=0.25, linewidth=0.5)
            plt.plot(gg1, yy, 'g-', alpha=0.25, linewidth=0.5)
    ggt = np.concatenate([gt,gt[::-1]])
    gg0 = np.concatenate([g0,g0[::-1]])
    gg1 = np.concatenate([g1,g1[::-1]])
    ax.plot(gg0, yy, 'r-', label='Initial')
    if plot_int:
        ggi = np.concatenate([gi,gi[::-1]])
        ax.plot(ggi, yy, 'c-', label='Intermediate') 
    ax.plot(gg1, yy, 'g-o', fillstyle='none', label='Final')
    ax.plot(ggt, yy, 'k-', label='Truth')
    ax.set_xlabel(r'$g_{' f'{i}' r'}$')
    ax.set_ylabel(r'$x_2$')
    ax.set_ylim([0., 2.])
    return ax

if plot_full_channel:
    plotfunc = plotg_double 
else:
    plotfunc = plotg

fig, ax = plt.subplots()
if plot_int:
    ax = plotfunc(ax, 1, g_t[:], g_0[:], g_1[:], g_i[:])
else:
    ax = plotfunc(ax, 1, g_t[:], g_0[:], g_1[:])
plt.legend()
    
if savefigs:
    fig.savefig('results_g1.pdf')

### Velocity

In [ ]:
yy = y.copy()
U0_0 = U_0[:,0]
if plot_int:
    U0_i = U_i[:,0]
U0_1 = U_1[:,0]
U0_t = U_t
lim = 1.0

if plot_full_channel:
    yy = np.concatenate([yy, yy+1.0])
    U0_0 = np.concatenate([U0_0, U0_0[::-1]])
    if plot_int:
        U0_i = np.concatenate([U0_i, U0_i[::-1]])
    U0_1 = np.concatenate([U0_1, U0_1[::-1]])
    U0_t = np.concatenate([U0_t, U0_t[::-1]])
    lim *= 2.0

plt.figure()
for isamp in range(nsamples):
    if plot_int:
        Hsi = H_i_samps[:, isamp]
        if plot_full_channel:
            Hsi = np.concatenate([Hsi, Hsi[::-1]])
        plt.plot(Hsi, yy, 'c-', alpha=0.25, linewidth=0.5)
    Hs0 = H_0_samps[:, isamp]
    if plot_full_channel:
        Hs0 = np.concatenate([Hs0, Hs0[::-1]])
    Hs1 = H_1_samps[:, isamp]
    if plot_full_channel:
        Hs1 = np.concatenate([Hs1, Hs1[::-1]])
    plt.plot(Hs0, yy, 'r-', alpha=0.25, linewidth=0.5)
    plt.plot(Hs1, yy, 'g-', alpha=0.25, linewidth=0.5)
plt.plot(U0_t, yy, 'k-', label='truth')
plt.plot(U0_0, yy, 'ro-', label='initial')
if plot_int:
    plt.plot(U0_i, yy, 'c-', label='intermediate')
plt.plot(U0_1, yy, 'go', fillstyle='none', label='final')
plt.xlabel(r'Velocity $U$')
plt.ylabel('$x_2$')
plt.legend()
plt.ylim([0.0, lim])
if savefigs:
    plt.savefig('results_U.pdf')
    
plt.xlim([0.0, 1.75])